In [160]:
import pypdf
from sklearn.cluster import KMeans, DBSCAN
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

geg_1 = "../data/Gebäudeenergiegesetz/2006415.pdf"
soli_1 = "../data/Abschaffung_des_Solidaritätszuschlags/1904898.pdf"

In [161]:
def extract_pages(path):
    reader = pypdf.PdfReader(path)
    return " ".join([page.extract_text() for page in reader.pages])

In [162]:
text_geg = extract_pages(geg_1)
text_soli = extract_pages(soli_1)

In [163]:
texts = [text_geg, text_soli]

In [164]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(texts)
similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

print(similarity_matrix)

[[1.         0.60626706]
 [0.60626706 1.        ]]


In [165]:
num_clusters = 2
km = KMeans(n_clusters=num_clusters, random_state=42)
clusters = km.fit_predict(tfidf_matrix)

In [166]:
import spacy

nlp = spacy.load("de_core_news_lg")

In [177]:
text_geg

'Deutscher Bundestag Drucksache 20/6415 \n20. Wahlperiode 18.04.2023 \n \nAntrag \nder Abgeordneten Marc Bernhard, Roger Beckamp, Carolin Bachmann, \nSebastian Münzenmaier, René Bochmann, Dr. Christina Baum, Dirk Brandes, \nMarcus Bühl, Dietmar Friedhoff, Dr. Malte Kaufmann, Dr. Michael Kaufmann, \nJörn König, Barbara Lenk, Edgar Naujok, Bernd Schattner, Jan Wenzel \nSchmidt, Dr. Dirk Spaniel und der Fraktion der AfD \nVerbot von Öl- und Gasheizungen verhindern – Priorisierung der \nWärmepumpen beenden  \nDer Bundestag wolle beschließen: \nI. Der Deutsche Bundestag stellt fest:  \nDer Wohnungsneubau in Deutschland kommt immer mehr zum Erliegen. Die Presse \nspricht mittlerweile von einer „Schockstarre“ 1. Auf den Weckruf durch die größte \ndeutsche Wohnungsbaugesellschaft Vonovia mit der Ankündigung in 2023 keine \nneuen Bauvorhaben zu beginnen2, reagierte die Politik nicht.  \nDer Referentenentwurf für die zweite Novellierung des Gebäudeenergiegesetzes aus \ndem Ministerium für Wirtsc

In [167]:
def lemma_no_punct_no_stop(text):
    return " ".join([token.lemma_ for token in nlp(text) if not (token.is_punct or token.is_stop)]).replace("\n", "")

In [168]:
lemma_text_geg = lemma_no_punct_no_stop(text_geg)
lemma_text_soli = lemma_no_punct_no_stop(text_soli)

print(lemma_text_geg)

deutsch Bundestag Drucksache 20 6415  20. Wahlperiode 18.04.2023   Antrag  abgeordneter Marc Bernhard Roger Beckamp Carolin Bachmann  Sebastian Münzenmaier René Bochmann Dr. Christina Baum Dirk Brandes  Marcus Bühl Dietmar Friedhoff Dr. Malte Kaufmann Dr. Michael Kaufmann  Jörn König Barbara Lenk Edgar Naujok Bernd Schattner Jan Wenzel  Schmidt Dr. Dirk Spaniel Fraktion AfD  Verbot Öl Gasheizung verhindern Priorisierung  Wärmepumpe beenden   Bundestag wollen beschließen  I. deutsch Bundestag stellen fest   Wohnungsneubau Deutschland erliegen Presse  sprechen mittlerweile Schockstarre 1. Weckruf groß  deutsch Wohnungsbaugesellschaft Vonovia Ankündigung 2023  Bauvorhaben beginnen2 reagieren Politik   Referentenentwurf Novellierung Gebäudeenergiegesetz  Ministerium Wirtschaft Klimaschutz zeigen Mal Bür-  ger vertrauen bestehend gesetzlich Regelung 2024  eingebaut Heizung mindestens 65 bereitgestellt Wärme  erneuerbar Energie erzeugen Heizung alt 30  betreiben 2045 Heizkessel fossil Brenns

In [169]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform([lemma_text_geg, lemma_text_soli])
similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

print(similarity_matrix)

[[1.         0.07759371]
 [0.07759371 1.        ]]


In [170]:
import numpy as np

def vectorize_spacy(text):
    word_vectors = [token.vector for token in nlp(text)]
    if not word_vectors:
        return np.zeros(word_vectors.vector_size)
    
    return np.mean(word_vectors[0], axis=0)

vectorize_spacy(lemma_text_geg)

0.1513122

## Splitting the texts into sections

In [171]:
print(text_geg)

Deutscher Bundestag Drucksache 20/6415 
20. Wahlperiode 18.04.2023 
 
Antrag 
der Abgeordneten Marc Bernhard, Roger Beckamp, Carolin Bachmann, 
Sebastian Münzenmaier, René Bochmann, Dr. Christina Baum, Dirk Brandes, 
Marcus Bühl, Dietmar Friedhoff, Dr. Malte Kaufmann, Dr. Michael Kaufmann, 
Jörn König, Barbara Lenk, Edgar Naujok, Bernd Schattner, Jan Wenzel 
Schmidt, Dr. Dirk Spaniel und der Fraktion der AfD 
Verbot von Öl- und Gasheizungen verhindern – Priorisierung der 
Wärmepumpen beenden  
Der Bundestag wolle beschließen: 
I. Der Deutsche Bundestag stellt fest:  
Der Wohnungsneubau in Deutschland kommt immer mehr zum Erliegen. Die Presse 
spricht mittlerweile von einer „Schockstarre“ 1. Auf den Weckruf durch die größte 
deutsche Wohnungsbaugesellschaft Vonovia mit der Ankündigung in 2023 keine 
neuen Bauvorhaben zu beginnen2, reagierte die Politik nicht.  
Der Referentenentwurf für die zweite Novellierung des Gebäudeenergiegesetzes aus 
dem Ministerium für Wirtschaft und Klimaschut

In [172]:
def split_by_sentence(text):
    doc = nlp(text)
    return [sent.text.strip() for sent in doc.sents if sent.text.strip()]

sentences_geg = split_by_sentence(text_geg)
sentences_soli = split_by_sentence(text_soli)

In [301]:
def remove_small_sentences(text_list):
    return [text for text in text_list if len(text.split()) > 10]

In [302]:
sentences_geg_no_small_sentences = remove_small_sentences(sentences_geg)
sentences_soli_no_small_sentences = remove_small_sentences(sentences_soli)

In [303]:
geg_list_lemmatized = [lemma_no_punct_no_stop(sentence) for sentence in sentences_geg_no_small_sentences]
soli_list_lemmatized = [lemma_no_punct_no_stop(sentence) for sentence in sentences_soli_no_small_sentences]

In [304]:
import pandas as pd
df = pd.DataFrame({
    "Sätze": geg_list_lemmatized + soli_list_lemmatized,
    "Cluster": ["Geg"] * len(geg_list_lemmatized) + ["Soli"] * len(soli_list_lemmatized)
})
df

,Sätze,Cluster
0,18.04.2023 Antrag abgeordneter Marc Bernhar...,Geg
1,Weckruf groß deutsch Wohnungsbaugesellschaft ...,Geg
2,Referentenentwurf Novellierung Gebäudeenergieg...,Geg
3,2024 eingebaut Heizung mindestens 65 bereitge...,Geg
4,Heizung alt 30 betreiben,Geg
...,...,...
116,2018 laut Bunderegierung mehrere Instit ution ...,Soli
117,Koalitionsvertrag festgeschrieben Verabredung...,Soli
118,Prof. Dr. Hanno Kube Lehrstuhl Steuerrecht Un...,Soli
119,sprechen ebenfalls selektiv Abschaffung besti...,Soli


In [314]:
def vectorize(sentence):
    return nlp(sentence).vector

In [315]:
df["Sätze_vektorisiert"] = df["Sätze"].apply(vectorize)

In [316]:
num_clusters = 2
km = KMeans(n_clusters=num_clusters, random_state=42)
clusters = km.fit_predict(df["Sätze_vektorisiert"])

ValueError: setting an array element with a sequence.

In [306]:
import pandas as pd

df["predicted_clusters"] = clusters

In [307]:
df

,Sätze,Cluster,predicted_clusters
0,18.04.2023 Antrag abgeordneter Marc Bernhar...,Geg,0
1,Weckruf groß deutsch Wohnungsbaugesellschaft ...,Geg,0
2,Referentenentwurf Novellierung Gebäudeenergieg...,Geg,0
3,2024 eingebaut Heizung mindestens 65 bereitge...,Geg,0
4,Heizung alt 30 betreiben,Geg,0
...,...,...,...
116,2018 laut Bunderegierung mehrere Instit ution ...,Soli,0
117,Koalitionsvertrag festgeschrieben Verabredung...,Soli,1
118,Prof. Dr. Hanno Kube Lehrstuhl Steuerrecht Un...,Soli,0
119,sprechen ebenfalls selektiv Abschaffung besti...,Soli,1


In [310]:
print(df.groupby(["predicted_clusters"]).size())


predicted_clusters
0    88
1    33
dtype: int64


In [313]:
for cluster_id in df["predicted_clusters"].unique():
    print(f"\n🔹 Cluster {cluster_id}:")
    print(df[df["predicted_clusters"] == cluster_id]["Sätze"].sample(10).tolist())  # Zeigt zufällige 5 Sätze



🔹 Cluster 0:
['ungerechtfertigt Ungleich Behandlung Ausland erzielt Einkünfte  inländisch Einkünfte Folge', 'Experte cha  Rakterisier Vorhaben unzulässig Eingriff Eigentumsrecht', 'Bundesrechnungshof unterschiedlich Behandlung inländisch ausländisch Einkünfte  sachgerecht beanstanden', 'Satz Satzweiss.com Print Web Software GmbH Mainzer Straße 116 66121 Saarbrücken www.satzweiss.com  Druck Printsystem GmbH Schafwäsche 1 3 71296 Heimsheim www.printsystem.de  Vertrieb Bundesanzeiger Verlag GmbH Postfach 10 05 34 50445 Köln Telefon 02 21 97 66 83 40 Fax 02 21 97 66 83 44 www.betrifft-gesetze.de  ISSN 0722 8333', 'Gesamtherstellung H. Heenemann GmbH Co. KG Buch Offsetdruckerei Bessemerstraße 83–91 12103 Berlin www.heenemann-druck.de  Vertrieb Bundesanzeiger Verlag GmbH Postfach 10 05 34 50445 Köln Telefon 02 21 97 66 83 40 Fax 02 21 97 66 83 44 www.bundesanzeiger-verlag.de  ISSN', 'hinzu Verschärfung Wohnungsnot Mieter  Miete zwangssaniert Wohnung zahlen', 'Per polyfluoriert Alkylsubstanz